In [ ]:
# %% Deep learning - Section 10.102
#    Code challenge 13: Adam with L2 regularisation

#    1) Start drom code from video 10.101 and 09.075
#    2) Modify code from 10.101 to use Adam with L2 regularisation
#    3) Use lr = 0.001 and batch_size = 32
#    4) Test a range of L2 regularisation (from 0 to 0.1)
#    5) Plot train and test accuracy against epochs for various L2 vals

# This code pertains a deep learning course provided by Mike X. Cohen on Udemy:
#   > https://www.udemy.com/course/deeplearning_x
# The "base" code in this repository is adapted (with very minor modifications)
# from code developed by the course instructor (Mike X. Cohen), while the
# "exercises" and the "code challenges" contain more original solutions and
# creative input from my side. If you are interested in DL (and if you are
# reading this statement, chances are that you are), go check out the course, it
# is singularly good.


In [ ]:
# %% Libraries and modules
import numpy               as np
import matplotlib.pyplot   as plt
import torch
import torch.nn            as nn
import seaborn             as sns
import copy
import torch.nn.functional as F
import pandas              as pd
import scipy.stats         as stats
import time

from torch.utils.data                 import DataLoader,TensorDataset
from sklearn.model_selection          import train_test_split
from google.colab                     import files
from torchsummary                     import summary
from IPython                          import display
from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('svg')


In [ ]:
# %% Create data

# General params
n_by_clust = 300
blurring   = 1

# Centroids
A = [1,1]
B = [5,1]
C = [4,4]

# Generate data
a = [ A[0]+np.random.randn(n_by_clust)*blurring, A[1]+np.random.randn(n_by_clust)*blurring ]
b = [ B[0]+np.random.randn(n_by_clust)*blurring, B[1]+np.random.randn(n_by_clust)*blurring ]
c = [ C[0]+np.random.randn(n_by_clust)*blurring, C[1]+np.random.randn(n_by_clust)*blurring ]

# Labels
labels_np = np.hstack(( np.zeros((n_by_clust)),
                        np.ones((n_by_clust)),
                        2*np.ones((n_by_clust)) ))

# Data matrix
data_np = np.hstack((a,b,c)).T

# Data into PyTorch tensors (long format for CCE)
data   = torch.tensor(data_np).float()
labels = torch.tensor(labels_np).long()


In [ ]:
# %% Plotting

phi = ( 1 + np.sqrt(5) ) / 2
fig = plt.figure(figsize=(6*phi,6))

plt.plot(data[np.where(labels==0)[0],0],data[np.where(labels==0)[0],1],'s',alpha=.75)
plt.plot(data[np.where(labels==1)[0],0],data[np.where(labels==1)[0],1],'o',alpha=.75)
plt.plot(data[np.where(labels==2)[0],0],data[np.where(labels==2)[0],1],'^',alpha=.75)

plt.title('Some clusters')
plt.xlabel('Dimension 1')
plt.ylabel('Dimension 2')
plt.grid()

plt.savefig('figure92_code_challenge_13.png')

plt.show()

files.download('figure92_code_challenge_13.png')


In [ ]:
# %% Split into train and test data

# Split with scikitlearn
train_data,test_data,train_labels,test_labels = train_test_split(data,labels,test_size=0.1)

# Convert into PyTorch datasets
train_data = TensorDataset(train_data,train_labels)
test_data  = TensorDataset(test_data,test_labels)

# Convert into DataLoader objects
test_loader  = DataLoader(test_data,batch_size=test_data.tensors[0].shape[0])


In [ ]:
# %% Create the model

def gen_model(optimizer_alg,learning_rate,L2_lambda):

    class model(nn.Module):
        def __init__(self):
            super().__init__()

            # Architecture
            self.input  = nn.Linear(2,8)
            self.hid1   = nn.Linear(8,8)
            self.output = nn.Linear(8,3)

        # Forward propagation
        def forward(self,x):
            x = F.relu(self.input(x))
            x = F.relu(self.hid1(x))
            x = self.output(x)

            return x

    # Model instance
    ANN = model()

    # Loss function and optimizer (get optimizer attribute)
    loss_fun  = nn.CrossEntropyLoss()
    opti_fun  = getattr( torch.optim,optimizer_alg )
    optimizer = opti_fun(ANN.parameters(),lr=learning_rate,weight_decay=L2_lambda)

    return ANN,loss_fun,optimizer


In [ ]:
# %% Check model

# Check weight decay
optim = gen_model('Adam',0.01,0.05)[2]
print(optim)


In [ ]:
# %% Function to train the model

# Epochs
num_epochs = 50

def train_model(optimizer_alg,learning_rate,L2_lambda):

    # Model instance
    ANN,loss_fun,optimizer = gen_model(optimizer_alg,learning_rate,L2_lambda)

    # Initialise
    losses    = []
    train_acc = []
    test_acc  = []

    # Epochs loop
    for epoch_i in range(num_epochs):

        # Train mode on
        ANN.train()

        # Initialise and loop over batches
        batch_losses = []
        batch_acc    = []

        for X,y in train_loader:

            # Forward propagation and loss
            yHat = ANN(X)
            loss = loss_fun(yHat,y)

            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Compute loss and accuracy from this batch
            batch_losses.append(loss.item())

            matches     = torch.argmax(yHat,axis=1) == y  # booleans
            matches_num = matches.float()                 # convert to numbers
            acc_percent = 100*torch.mean(matches_num)     # average and percent
            batch_acc.append(acc_percent)

        # Average train accuracy and losses from batches
        train_acc.append(np.mean(batch_acc))
        losses.append(np.mean(batch_losses))

        # Test accuracy (turn autograd off)
        ANN.eval()
        X,y = next(iter(test_loader))
        with torch.no_grad():
            yHat = ANN(X)
        test_acc.append( 100*torch.mean((torch.argmax(yHat,axis=1)==y).float()) )

    return train_acc,test_acc,losses,ANN


In [ ]:
# %% Run experiment

# Use only Andam and lr=0.001 for this code challenge (takes ~1 mins)
L2_lambda      = np.linspace(0,0.1,6)
batch_sizes    = [8,32,128]

accuracy_train = np.zeros((num_epochs,len(L2_lambda),len(batch_sizes)))
accuracy_test  = np.zeros((num_epochs,len(L2_lambda),len(batch_sizes)))

for i,L2 in enumerate(L2_lambda):
    for k,batch_size in enumerate(batch_sizes):

        train_loader = DataLoader(train_data,batch_size=batch_size,shuffle=True,drop_last=True)
        train_acc,test_acc,losses,ANN = train_model('Adam',0.001,L2)

        accuracy_train[:,i,k] = np.array(train_acc)
        accuracy_test[:,i,k]  = np.array(test_acc)


In [ ]:
# %% Functions for 1D smoothing filter

# Improved for edge effects - adaptive window
def smooth_adaptive(x,k):
    smoothed = np.zeros_like(x)
    half_k   = k // 2

    for i in range(len(x)):
        start       = max(0, i-half_k)
        end         = min(len(x), i+half_k + 1)
        smoothed[i] = np.mean(x[start:end])

    return smoothed


In [ ]:
# %% Plotting

phi = ( 1 + np.sqrt(5) ) / 2
fig,ax = plt.subplots(1,2,figsize=(1.5*6*phi,6))

cmaps = plt.cm.plasma(np.linspace(.1,.9,len(L2_lambda)))
for i in range(len(L2_lambda)):
    ax[0].plot(smooth_adaptive(accuracy_train[:,i,1],5),color=cmaps[i])
    ax[1].plot(smooth_adaptive(accuracy_test[:,i,1],5),color=cmaps[i])

ax[0].set_title('Train accuracy')
ax[1].set_title('Test accuracy')

# Make the legend easier to read
leglabels = [fun for i,fun in enumerate(L2_lambda)]

# Common features
for i in range(2):
    ax[i].legend(leglabels)
    ax[i].set_xlabel('Epoch')
    ax[i].set_ylabel('Accuracy (%)')
    ax[i].set_ylim([30,101])
    ax[i].grid()

plt.suptitle('Batch size = 32')

plt.savefig('figure93_code_challenge_13.png')

plt.show()

files.download('figure93_code_challenge_13.png')
